## DEMO:
obj: Mini demo to launch a studies with multiple parameters...

In [4]:
# coding: utf-8
# system import
import pickle
import numpy as np
import os.path as osp
import scipy.fftpack as pfft
from pprint import pprint

# module import
from meri.metric import compute_ssim, compute_snr, compute_psnr, compute_nrmse
from meri.grid_search import grid_search
import pisap
from pisap.base.utils import l2_normalize, convert_mask_to_locations
from pisap.numerics.gradient import Grad2DAnalyse
from pisap.numerics.fourier import FFT
from pisap.numerics.linears import *
from pisap.numerics.reconstruct import sparse_rec_fista

### Load and preprocess the image to reconstruct:

In [5]:
impath = "M31_128.fits"
im = pisap.io.load(impath).data.astype('float64')
maskfile = "mask25_sig40.fits"
maskpath = "mask25_sig40.fits"
mask = pisap.io.load(maskpath).data
imfft = pfft.fft2(im)
dirty_im = pfft.ifft2(imfft)

### Declare the parameters:

In [6]:
ft_cls = FFT(convert_mask_to_locations(pfft.ifftshift(mask)), im.shape[0])
list_params = {
    'data':imfft,
    'gradient_cls':Grad2DAnalyse,
    'gradient_kwargs':{"ft_cls": ft_cls},
    'linear_cls': [MallatWaveletTransform7_9Filters, MeyerWaveletsCompactInFourierSpace],
    'linear_kwargs':[{"maxscale": 3}, {"maxscale": 4}, {"maxscale": 5}, {"maxscale": 6},],
    'max_nb_of_iter': 25,
    'mu': [5.0e-7, 1.0e-6, 2.0e-6, 3.0e-6, 5.0e-6],
    'verbose':0,
    'report':False,
}
metrics = [compute_ssim, compute_snr, compute_psnr, compute_nrmse]
metrics_direction= [False, False, False, True]

pprint(list_params)

{'data': array([[ 373.85763741  +0.j        , -281.61535147 +31.54498208j,
         134.31091044 -52.66783129j, ...,  -59.70542242 -17.39332953j,
         134.31091044 +52.66783129j, -281.61535147 -31.54498208j],
       [-217.83774541+187.43616597j,  143.94249233-174.64818511j,
         -69.71641148+122.9013679j , ...,   34.38468778 -40.77715704j,
         -68.06646021 +88.03717727j,  172.50048014-162.75140087j],
       [  -9.49499651-156.68525062j,   13.26131491+130.16628935j,
          10.95580187 -77.2631067j , ...,    5.22450331 +23.99141585j,
         -33.15696347-102.5408075j ,   12.36127455+161.55459253j],
       ..., 
       [  53.00851145 -47.83522715j,  -36.59426213 +48.94657823j,
          13.96686551 -29.77100384j, ...,    3.24409458 +30.26874116j,
         -11.77198246 -45.46551581j,  -24.54364454 +56.80855794j],
       [  -9.49499651+156.68525062j,   12.36127455-161.55459253j,
         -33.15696347+102.5408075j , ...,  -12.05121362 -22.18889378j,
          10.95580187 +77

### Create the report:

In [7]:
print("Running the grid_search...")
report = grid_search(sparse_rec_fista, list_params, metrics, metrics_direction,
                     im, n_jobs=-1, verbose=5)

Running the grid_search...
Running 4 metrics for 40 candidates, totalling 40 reconstruction and 160 metrics computations


/home/hc252523/COSMIC/src/pisap/pisap/base/dictionary.py:540: ComplexWarning: Casting complex values to real discards the imaginary part
  data = self._data.reshape(self.native_image_shape).astype(float)
/home/hc252523/COSMIC/src/pisap/pisap/base/dictionary.py:540: ComplexWarning: Casting complex values to real discards the imaginary part
  data = self._data.reshape(self.native_image_shape).astype(float)
/home/hc252523/COSMIC/src/pisap/pisap/base/dictionary.py:540: ComplexWarning: Casting complex values to real discards the imaginary part
  data = self._data.reshape(self.native_image_shape).astype(float)
/home/hc252523/COSMIC/src/pisap/pisap/base/dictionary.py:540: ComplexWarning: Casting complex values to real discards the imaginary part
  data = self._data.reshape(self.native_image_shape).astype(float)
/home/hc252523/COSMIC/src/pisap/pisap/base/dictionary.py:540: ComplexWarning: Casting complex values to real discards the imaginary part
  data = self._data.reshape(self.native_image_s

### Save the report:

In [8]:
filename = "report.pkl"
print("Inspect the generated pickle file {0}"
                    " to see the corresponding parameters...".format(filename))
print("Pickeling the report class in '{0}'...".format(filename))
with open(filename, 'w') as pfile:
    pickle.dump(report, pfile)

Inspect the generated pickle file report.pkl to see the corresponding parameters...
Pickeling the report class in 'report.pkl'...
